## Deliverable 2. Create a Customer Travel Destinations Map.

In [4]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [15]:
# 1. Import the WeatherPy_database.csv file.
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,70.88,63,74,21.34,broken clouds
1,1,Zyryanka,RU,65.7500,150.8500,47.71,94,100,6.08,overcast clouds
2,2,Santa Rosa,PH,14.3122,121.1114,76.98,96,100,4.56,moderate rain
3,3,Codrington,AU,-38.2667,141.9667,41.68,97,11,7.07,few clouds
4,4,Nyrob,RU,60.7330,56.7202,65.07,68,84,2.19,broken clouds


In [16]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 85


In [17]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Santa Rosa,PH,14.3122,121.1114,76.98,96,100,4.56,moderate rain
12,12,Hithadhoo,MV,-0.6000,73.0833,82.54,70,100,9.86,overcast clouds
20,20,Lubao,PH,14.9167,120.5500,78.87,92,99,1.86,moderate rain
21,21,Butaritari,KI,3.0707,172.7902,81.81,75,27,13.76,scattered clouds
26,26,Vaini,TO,-21.2000,-175.2000,75.36,94,100,4.61,overcast clouds
29,29,Alugan,PH,12.2188,125.4808,77.65,89,100,9.78,overcast clouds
31,31,Sao Filipe,CV,14.8961,-24.4956,77.95,79,58,9.08,broken clouds
34,34,Hami,CN,42.8000,93.4500,84.51,37,70,4.97,broken clouds
35,35,Atuona,PF,-9.8000,-139.0333,76.57,72,67,15.93,broken clouds
37,37,Limbang,MY,4.7500,115.0000,77.11,98,100,3.56,overcast clouds


In [18]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                186
City                   186
Country                186
Lat                    186
Lng                    186
Max Temp               186
Humidity               186
Cloudiness             186
Wind Speed             186
Current Description    186
dtype: int64

In [21]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df= preferred_cities_df

In [22]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Santa Rosa,PH,76.98,moderate rain,14.3122,121.1114,
12,Hithadhoo,MV,82.54,overcast clouds,-0.6000,73.0833,
20,Lubao,PH,78.87,moderate rain,14.9167,120.5500,
21,Butaritari,KI,81.81,scattered clouds,3.0707,172.7902,
26,Vaini,TO,75.36,overcast clouds,-21.2000,-175.2000,
29,Alugan,PH,77.65,overcast clouds,12.2188,125.4808,
31,Sao Filipe,CV,77.95,broken clouds,14.8961,-24.4956,
34,Hami,CN,84.51,broken clouds,42.8000,93.4500,
35,Atuona,PF,76.57,broken clouds,-9.8000,-139.0333,
37,Limbang,MY,77.11,overcast clouds,4.7500,115.0000,


In [24]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [36]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Santa Rosa,PH,76.98,moderate rain,14.3122,121.1114,Erlyn’s Sta Rosa City Home
12,Hithadhoo,MV,82.54,overcast clouds,-0.6000,73.0833,Scoop Guest House
20,Lubao,PH,78.87,moderate rain,14.9167,120.5500,Hacienda Gracia
21,Butaritari,KI,81.81,scattered clouds,3.0707,172.7902,Isles Sunset Lodge
26,Vaini,TO,75.36,overcast clouds,-21.2000,-175.2000,Keleti Beach Resort


In [43]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [44]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
# Source: https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html

info_box_template = """
<dl>
<dt>Hotel</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.

clean_hotel_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")

hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [45]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))